In [1]:
from datasets import load_dataset, DatasetDict, Dataset

In [2]:
# Load the CNN/Daily Mail dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')

Found cached dataset cnn_dailymail (C:/Users/basel/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
from datasets import load_dataset
import pandas as pd

# Load the CNN/Daily Mail dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')

# Create a DataFrame with Original Articles and Original Summary
cnn_df = pd.DataFrame({
    'Original Articles': dataset['train']['article'],
    'Original Summary': dataset['train']['highlights']
})
cnn_mini_df = cnn_df[:1500]


Found cached dataset cnn_dailymail (C:/Users/basel/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

In [34]:
cnn_mini_df['Original Summary'][1]

'Mentally ill inmates in Miami are housed on the "forgotten floor"\nJudge Steven Leifman says most are there as a result of "avoidable felonies"\nWhile CNN tours facility, patient shouts: "I am the son of the president"\nLeifman says the system is unjust and he\'s fighting for change .'

In [18]:
from summarization_algorithm import *
from preprocessing_algorithms import *

In [24]:
sentences , filtered_sentences = process_one_column_df(cnn_mini_df['Original Articles'],True,True)

100%|██████████| 1500/1500 [36:28<00:00,  1.46s/it]


In [29]:
# Save the mini DataFrame to a text file in JSON format
with open('cnn_mini_df.txt', 'w') as f:
    f.write(cnn_mini_df.to_json())

# Load the mini DataFrame from the text file in JSON format
with open('cnn_mini_df.txt', 'r') as f:
    cnn_mini_df_json = f.read()

cnn_mini_df_loaded = pd.read_json(cnn_mini_df_json)

# Print the mini DataFrame
print(cnn_mini_df_loaded)

                                      Original Articles   
0     LONDON, England (Reuters) -- Harry Potter star...  \
1     Editor's note: In our Behind the Scenes series...   
2     MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...   
3     WASHINGTON (CNN) -- Doctors removed five small...   
4     (CNN)  -- The National Football League has ind...   
...                                                 ...   
1495  WASHINGTON (CNN) -- A court Thursday rejected ...   
1496  ORLANDO, Florida (CNN) -- The meter reader who...   
1497  (CNN) -- Former first lady Barbara Bush was mo...   
1498  DENVER, Colorado (CNN)  -- Investigators will ...   
1499  WASHINGTON (CNN) -- Iraq's smaller religious g...   

                                       Original Summary  
0     Harry Potter star Daniel Radcliffe gets £20M f...  
1     Mentally ill inmates in Miami are housed on th...  
2     NEW: "I thought I was going to die," driver sa...  
3     Five small polyps found during procedure; "non...  
4

In [44]:
def summarize_with(list_of_filtered_articles, list_of_articles, summary_algorithm, size=0.2):
    rows = len(list_of_articles)
    summarized_text = []
    for row in tqdm(range(rows)):
        sentences = list_of_articles[row]
        filtered_sentences = list_of_filtered_articles[row]
        summary_size = int(size * len(sentences)) if 0 < size < 1 else int(size)
        summary = summary_algorithm(filtered_sentences, sentences, size=summary_size)
        summarized_text.append(summary)
    summary_df = pd.DataFrame(summarized_text, columns=[f'{summary_algorithm.__name__} summary'])
    return summary_df


In [45]:
luhn_summary = summarize_with(filtered_sentences,sentences,luhn_algorithm,0.2)

100%|██████████| 1500/1500 [00:00<00:00, 4832.82it/s]


In [46]:
luhn_summary

,luhn_algorithm summary
0,"LONDON, England (Reuters) -- Harry Potter star..."
1,Mentally ill people often won't do what they'r...
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we..."
3,He planned to have lunch at Camp David and hav...
4,Vick said he would plead guilty to one count o...
...,...
1495,"The District of Columbia Court of Appeals ""rul..."
1496,Sheriff's spokesman Carlos Padilla said last w...
1497,(CNN) -- Former first lady Barbara Bush was mo...
1498,All 115 people aboard the Continental Airlines...


In [54]:
from efficiency_scores import *

In [55]:
luhn_scores = rouge_scores_df(cnn_mini_df,luhn_summary)

100%|██████████| 1500/1500 [38:10<00:00,  1.53s/it]


In [59]:
luhn_scores

,rouge1_fmeasure,rouge1_precision,rouge1_recall,rouge2_fmeasure,rouge2_precision,rouge2_recall,rougeL_fmeasure,rougeL_precision,rougeL_recall,rougeLsum_fmeasure,rougeLsum_precision,rougeLsum_recall
0,tensor(0.2500),tensor(0.1525),tensor(0.6923),tensor(0.1776),tensor(0.1080),tensor(0.5000),tensor(0.2315),tensor(0.1412),tensor(0.6410),tensor(0.2407),tensor(0.1469),tensor(0.6667)
1,tensor(0.2207),tensor(0.1320),tensor(0.6735),tensor(0.0741),tensor(0.0442),tensor(0.2292),tensor(0.1338),tensor(0.0800),tensor(0.4082),tensor(0.1338),tensor(0.0800),tensor(0.4082)
2,tensor(0.1164),tensor(0.0684),tensor(0.3902),tensor(0.0220),tensor(0.0129),tensor(0.0750),tensor(0.0655),tensor(0.0385),tensor(0.2195),tensor(0.1091),tensor(0.0641),tensor(0.3659)
3,tensor(0.1266),tensor(0.0746),tensor(0.4167),tensor(0.0256),tensor(0.0150),tensor(0.0870),tensor(0.0506),tensor(0.0299),tensor(0.1667),tensor(0.0886),tensor(0.0522),tensor(0.2917)
4,tensor(0.1183),tensor(0.0669),tensor(0.5122),tensor(0.0397),tensor(0.0224),tensor(0.1750),tensor(0.0676),tensor(0.0382),tensor(0.2927),tensor(0.1070),tensor(0.0605),tensor(0.4634)
...,...,...,...,...,...,...,...,...,...,...,...,...
1495,tensor(0.1294),tensor(0.0774),tensor(0.3939),tensor(0.0101),tensor(0.0060),tensor(0.0312),tensor(0.0697),tensor(0.0417),tensor(0.2121),tensor(0.0796),tensor(0.0476),tensor(0.2424)
1496,tensor(0.2927),tensor(0.1818),tensor(0.7500),tensor(0.1148),tensor(0.0711),tensor(0.2979),tensor(0.1626),tensor(0.1010),tensor(0.4167),tensor(0.2520),tensor(0.1566),tensor(0.6458)
1497,tensor(0.3019),tensor(0.2462),tensor(0.3902),tensor(0.1923),tensor(0.1562),tensor(0.2500),tensor(0.2830),tensor(0.2308),tensor(0.3659),tensor(0.2830),tensor(0.2308),tensor(0.3659)
1498,tensor(0.1799),tensor(0.1037),tensor(0.6757),tensor(0.0507),tensor(0.0292),tensor(0.1944),tensor(0.1079),tensor(0.0622),tensor(0.4054),tensor(0.1655),tensor(0.0954),tensor(0.6216)


In [56]:
luhn_avg = df_avg_by_column(luhn_scores)

In [58]:
luhn_avg

rouge1_fmeasure        tensor(0.2153)
rouge1_precision       tensor(0.1442)
rouge1_recall          tensor(0.5482)
rouge2_fmeasure        tensor(0.0767)
rouge2_precision       tensor(0.0519)
rouge2_recall          tensor(0.1905)
rougeL_fmeasure        tensor(0.1361)
rougeL_precision       tensor(0.0916)
rougeL_recall          tensor(0.3441)
rougeLsum_fmeasure     tensor(0.1845)
rougeLsum_precision    tensor(0.1235)
rougeLsum_recall       tensor(0.4713)
dtype: object

In [57]:
luhn_scores.to_csv('cnn_luhn.csv')

In [31]:
cnn_mini_df_loaded.keys()

Index(['Original Articles', 'Original Summary'], dtype='object')

In [36]:
import pandas as pd

# Calculate the number of summary sentences for each article
num_summary_sentences = [len(article.split('.')) for article in cnn_mini_df['Original Summary']]

# Calculate the average number of summary sentences per article
avg_num_summary_sentences = sum(num_summary_sentences) / len(num_summary_sentences)

# Print the result
print(f"The average length of the original summaries over the entire dataset is {avg_num_summary_sentences:.2f} sentences.")


The average length of the original summaries over the entire dataset is 5.06 sentences.


In [38]:

# Get the number of summary sentences for each article

# Calculate the number of summary sentences for each article
summary_lengths = [len(article.split('.')) for article in cnn_mini_df['Original Summary']]
max_summary_length = max(summary_lengths)
min_summary_length = min(summary_lengths)

# Get the maximum and minimum length of summary sentences in the dataset
max_summary_length = max(summary_lengths)
min_summary_length = min(summary_lengths)

# Print the results
print("Maximum summary length:", max_summary_length)
print("Minimum summary length:", min_summary_length)


Maximum summary length: 13
Minimum summary length: 2
